In [1]:
import numpy as np
from scipy.optimize import minimize

# Problem parameters
num_facilities = 3
num_customers = 5
num_scenarios = 100

# Fixed costs for opening facilities
fixed_costs = np.array([100000, 120000, 90000])

# Transportation costs from facilities to customers
trans_costs = np.array([
    [10, 15, 20, 25, 30],
    [12, 14, 18, 22, 26],
    [8, 16, 24, 32, 40]
])

# Facility capacities
capacities = np.array([5000, 6000, 4500])

# Generate random demand scenarios
np.random.seed(42)
demand_mean = 1000
demand_std = 200
demand_scenarios = np.random.normal(demand_mean, demand_std, (num_scenarios, num_customers))
demand_scenarios = np.clip(demand_scenarios, 0, None)  # Ensure non-negative demand

# Scenario probabilities
scenario_probs = np.ones(num_scenarios) / num_scenarios

# Progressive Hedging parameters
rho = 1000  # Penalty parameter
max_iterations = 100
epsilon = 1e-6  # Convergence tolerance

def scenario_subproblem(x, w, z, s):
    facility_open = x[:num_facilities]
    distribution = x[num_facilities:].reshape((num_facilities, num_customers))
    
    fixed_cost = np.dot(fixed_costs, facility_open)
    transport_cost = np.sum(trans_costs * distribution)
    penalty_cost = np.dot(w, x - z) + (rho/2) * np.sum((x - z)**2)
    
    total_cost = fixed_cost + transport_cost + penalty_cost
    
    return total_cost

def scenario_constraints(x, s):
    facility_open = x[:num_facilities]
    distribution = x[num_facilities:].reshape((num_facilities, num_customers))
    
    # Capacity constraints
    capacity_cons = [np.sum(distribution[i]) - capacities[i] * facility_open[i] for i in range(num_facilities)]
    
    # Demand satisfaction constraints
    demand_cons = [np.sum(distribution[:, j]) - demand_scenarios[s, j] for j in range(num_customers)]
    
    return np.concatenate([capacity_cons, demand_cons])

def progressive_hedging():
    # Initialize
    x = {s: np.random.rand(num_facilities + num_facilities * num_customers) for s in range(num_scenarios)}
    w = {s: np.zeros_like(x[s]) for s in range(num_scenarios)}
    z = np.mean([x[s] for s in range(num_scenarios)], axis=0)
    
    for iteration in range(max_iterations):
        # Solve scenario subproblems
        for s in range(num_scenarios):
            result = minimize(
                scenario_subproblem,
                x[s],
                args=(w[s], z, s),
                method='SLSQP',
                constraints={'type': 'ineq', 'fun': lambda x: -scenario_constraints(x, s)},
                bounds=[(0, 1)] * num_facilities + [(0, None)] * (num_facilities * num_customers)
            )
            x[s] = result.x
        
        # Compute average solution
        z_new = np.mean([x[s] for s in range(num_scenarios)], axis=0)
        
        # Update price variables
        for s in range(num_scenarios):
            w[s] += rho * (x[s] - z_new)
        
        # Check convergence
        if np.max([np.max(np.abs(x[s] - z_new)) for s in range(num_scenarios)]) < epsilon:
            break
        
        z = z_new
    
    return z

# Run the algorithm
solution = progressive_hedging()

# Extract and print results
facility_open = solution[:num_facilities]
distribution = solution[num_facilities:].reshape((num_facilities, num_customers))

print("Facilities to open:")
for i in range(num_facilities):
    if facility_open[i] > 0.5:
        print(f"Facility {i+1}")

print("\nAverage distribution:")
for i in range(num_facilities):
    for j in range(num_customers):
        if distribution[i, j] > 0:
            print(f"From Facility {i+1} to Customer {j+1}: {distribution[i, j]:.2f}")

# Calculate and print total cost
total_fixed_cost = np.dot(fixed_costs, facility_open)
total_transport_cost = np.sum(trans_costs * distribution)
print(f"\nTotal Fixed Cost: {total_fixed_cost:.2f}")
print(f"Total Transport Cost: {total_transport_cost:.2f}")
print(f"Total Cost: {total_fixed_cost + total_transport_cost:.2f}")


Facilities to open:

Average distribution:
From Facility 1 to Customer 1: 1.78
From Facility 1 to Customer 2: 1.36
From Facility 1 to Customer 3: 1.66
From Facility 1 to Customer 4: 1.37
From Facility 1 to Customer 5: 1.63
From Facility 2 to Customer 1: 2.47
From Facility 2 to Customer 2: 2.42
From Facility 2 to Customer 3: 1.67
From Facility 2 to Customer 4: 0.96
From Facility 2 to Customer 5: 0.99
From Facility 3 to Customer 1: 2.26
From Facility 3 to Customer 2: 2.56
From Facility 3 to Customer 3: 2.12
From Facility 3 to Customer 4: 0.64
From Facility 3 to Customer 5: 0.93

Total Fixed Cost: 425.28
Total Transport Cost: 462.93
Total Cost: 888.22


In [2]:
solution.shape

(18,)